# PCA on Covariates Simulations Notebook

In [1]:
# Packages
import numpy as np
import pandas as pd
import seaborn as sns
from pca import pca
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import math
from tqdm import tqdm

In [2]:
# Supressing Output
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

**My main changes and what I want people to review in the code block below:**
    
1) I added an IV section

2) I added a new thing where we take half of the mismeasured covariate values and then use them as the exponent to raise e to (e.g. z4 = e**z4)

3) i got rid of the 1,000 value for beta1 and beta2

4) i got rid of the p = 10 value

**Comments on those changes:**

1) i'm curious if i did this right

2) i did this because of bonhomme's feedback about where PCA might outperform taking the simple avg. we've gotten some tentative signs that PCA does do better than the avg when you stretch out he observations

3) , 4) it seemed like we didn't really need those values

In [3]:
# Data frame to store output
output = pd.DataFrame()

# 2,000 observations
N = 2000

# Loop over combinations of: betas, covariances between variables, p numbers of parameters
for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,20,50]:
                # Select only the scenarios we actually want to run by default - betas of 1, covariances of 0.5, p of 5... require three of these conditions to be satisfied to run.
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    # Run with and without transformations
                    for exp_of_var in ['yes','no']:
                        # 1000 simulations
                        for k in tqdm(range(1)):
                            # Initialize Lists to store coef values for all five methods and the true coef
                            pca_coef = []
                            mismeasured_coef = []
                            mismeasured_allvar_coef = []
                            mismeasured_avg_coef = []
                            iv_coef = []
                            true_val_coef =[]

                            # Create variables
                            vars_mean = [0,0,0]
                            vars_cov = np.array([[1,covariance,0],
                                                 [covariance,1,0],
                                                 [0,0,1]])
                            # Producing 3 variables: x for the variable of interest, the true Z covariate, the random error
                            vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                            vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                            # Create measurement errors for each of the p measurements of the covariates- mean zero and variance one
                            errors_mean = np.zeros(p)
                            errors_cov = np.zeros((p,p))
                            for i in range(p):
                                for j in range(p):
                                    if i == j:
                                        errors_cov[i,j] = 1

                            errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                            # Column labels for Z variables (covariates variables mismeasured)
                            z_vars = []
                            for i in range(p):
                                z_vars.append('z'+str(i+1))
                            # Add errors to the true_z to get mismeasured values
                            mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                            for i in mismeasured_z.columns:
                                mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                            # Take e to the power of the values for half of the measurements if log_of_var is true
                            if exp_of_var == 'yes':
                                mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):] =np.exp(mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):])
                                
                            # Do feature scaling (normalize to mean 0 and variance 1) for the mismeasured z
                            # Note that x and y are already normalized by construction
                            scaled_mismeasured_z = mismeasured_z.copy()
                            for i in mismeasured_z.columns:
                                scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                            # Suppress output
                            with suppress_stdout():
                                # Use PCA on the mismeasured values
                                pca_model = pca()
                                pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                                pca_z = pca_results['PC']['PC1']

                            # NOTE: in non-pca cases, no need to rescale or normalize since mismeasured variables and x and y have mean 0 and sd 1

                            # Average mismeasured variables:
                            vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                            # Add relevant variables to vars_ dataframe
                            vars_[mismeasured_z.columns] = mismeasured_z
                            vars_['pca_z'] = pca_z

                            # Single mismeasured covariate results
                            model_mismeasured = sm.OLS(vars_['y'],vars_[['x','z1']])
                            results_mismeasured = model_mismeasured.fit()
                            mismeasured_coef.append(results_mismeasured.params[0])

                            # All Variables Mismeasured Results
                            # Create full list of items to include in regression
                            tot_vars = ['x']
                            tot_vars.extend(z_vars)
                            model_mismeasured_allvar = sm.OLS(vars_['y'],vars_[tot_vars])
                            results_mismeasured_allvar = model_mismeasured_allvar.fit()
                            mismeasured_allvar_coef.append(results_mismeasured_allvar.params[0])

                            # Average Mismeasured Variables Results
                            model_mismeasured_avg = sm.OLS(vars_['y'],vars_[['x','avg_mismeasured_z']])
                            results_mismeasured_avg = model_mismeasured_avg.fit()
                            mismeasured_avg_coef.append(results_mismeasured_avg.params[0])

                            # PCA Results
                            model_pca = sm.OLS(vars_['y'],vars_[['x','pca_z']])
                            results_pca = model_pca.fit()
                            pca_coef.append(results_pca.params[0])

                            # Instrumental Variables Results
                            # Instrument z1 on the other items in the mismeasured df
                            z_string = mismeasured_z.columns[1]
                            for z in mismeasured_z.columns[2:]:
                                z_string = z_string + ' + ' + z
                            # Create the predicted value of z1
                            vars_['pred_z1'] = smf.ols("z1 ~ " + z_string, data = mismeasured_z).fit().predict()
                            iv_results = smf.ols("y ~ x + pred_z1 -1", data = vars_).fit()
                            iv_coef.append(iv_results.params[0])

                            # True Results
                            model_true = sm.OLS(vars_['y'],vars_[['x','true_z']])
                            results_true = model_true.fit()
                            true_val_coef.append(results_true.params[0])

                            # Output Findings
                            new_output = pd.DataFrame()
                            new_output['mismeasured_coef'] = mismeasured_coef
                            new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                            new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                            new_output['pca_coef'] = pca_coef
                            new_output['true_val_coef'] = true_val_coef
                            new_output['iv_coef'] = iv_coef
                            new_output['covariance'] = vars_cov[0][1]
                            new_output['beta1'] = beta1
                            new_output['beta2'] = beta2
                            new_output['p'] = p
                            new_output['exp_of_var'] = exp_of_var
                            output = output.append(new_output)

output

100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.392676,0.297068,0.482867,0.347941,0.126243,0.362740,0.5,0.1,1.0,5,yes
0,0.367448,0.170904,0.171895,0.171929,0.053409,0.196630,0.5,0.1,1.0,5,no
0,0.994095,0.983749,0.992905,0.973718,0.975325,0.979110,0.5,1.0,0.1,5,yes
0,0.997181,0.988845,0.989324,0.989386,0.976645,0.994704,0.5,1.0,0.1,5,no
0,0.306442,0.431522,0.210874,0.386974,1.108455,0.356959,-0.9,1.0,1.0,5,yes
0,0.217245,0.477989,0.477303,0.477019,0.952746,0.422875,-0.9,1.0,1.0,5,no
0,0.688088,0.809595,0.591318,0.787940,0.991823,0.753843,-0.5,1.0,1.0,5,yes
0,0.724498,0.920783,0.919703,0.918035,1.012534,0.904684,-0.5,1.0,1.0,5,no
0,0.981324,0.981242,1.000192,0.989517,0.992666,0.983436,0.0,1.0,1.0,5,yes
0,1.005180,1.001407,1.000276,1.000394,1.009075,1.000825,0.0,1.0,1.0,5,no


In [4]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.392676,0.297068,0.482867,0.347941,0.126243,0.362740,0.5,0.1,1.0,5,yes
0,0.367448,0.170904,0.171895,0.171929,0.053409,0.196630,0.5,0.1,1.0,5,no
0,0.994095,0.983749,0.992905,0.973718,0.975325,0.979110,0.5,1.0,0.1,5,yes
0,0.997181,0.988845,0.989324,0.989386,0.976645,0.994704,0.5,1.0,0.1,5,no
0,0.306442,0.431522,0.210874,0.386974,1.108455,0.356959,-0.9,1.0,1.0,5,yes
0,0.217245,0.477989,0.477303,0.477019,0.952746,0.422875,-0.9,1.0,1.0,5,no
0,0.688088,0.809595,0.591318,0.787940,0.991823,0.753843,-0.5,1.0,1.0,5,yes
0,0.724498,0.920783,0.919703,0.918035,1.012534,0.904684,-0.5,1.0,1.0,5,no
0,0.981324,0.981242,1.000192,0.989517,0.992666,0.983436,0.0,1.0,1.0,5,yes
0,1.005180,1.001407,1.000276,1.000394,1.009075,1.000825,0.0,1.0,1.0,5,no


In [5]:
#output.to_csv('C://Users//paulo//Documents//Econometrics and ML//Final Project//n_2000_results.csv')

In [6]:
pd.set_option('display.max_rows', 10)

In [7]:
output['pca_ape'] = (output['pca_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_coef_ape'] = (output['mismeasured_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_allvar_coef_ape'] = (output['mismeasured_allvar_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_avg_coef_ape'] = (output['mismeasured_avg_coef'] - output['beta1']).abs()/output['beta1']
output['iv_coef_ape'] = (output['iv_coef'] - output['beta1']).abs()/output['beta1']

In [8]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.217245,0.477989,0.477303,0.477019,0.952746,0.422875,0.522981,0.782755,0.522011,0.522697,0.577125
1,-0.9,1.0,1.0,5,yes,0.306442,0.431522,0.210874,0.386974,1.108455,0.356959,0.613026,0.693558,0.568478,0.789126,0.643041
2,-0.5,1.0,1.0,5,no,0.724498,0.920783,0.919703,0.918035,1.012534,0.904684,0.081965,0.275502,0.079217,0.080297,0.095316
3,-0.5,1.0,1.0,5,yes,0.688088,0.809595,0.591318,0.787940,0.991823,0.753843,0.212060,0.311912,0.190405,0.408682,0.246157
4,0.0,1.0,1.0,5,no,1.005180,1.001407,1.000276,1.000394,1.009075,1.000825,0.000394,0.005180,0.001407,0.000276,0.000825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,3.913515,2.907110,4.573856,2.977885,1.040922,3.442474,1.977885,2.913515,1.907110,3.573856,2.442474
18,0.5,10.0,1.0,5,no,10.298414,10.105304,10.105292,10.105762,9.997272,10.121088,0.010576,0.029841,0.010530,0.010529,0.012109
19,0.5,10.0,1.0,5,yes,10.278445,10.178115,10.397776,10.237330,9.982137,10.243914,0.023733,0.027845,0.017811,0.039778,0.024391
20,0.9,1.0,1.0,5,no,1.813492,1.456942,1.457083,1.457555,1.000043,1.501181,0.457555,0.813492,0.456942,0.457083,0.501181


In [9]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.9,1.0,1.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.5,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.5,1.0,1.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,0.5,10.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.5,10.0,1.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,0.9,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,1.273635,1.184514,1.324573,1.188763,0.990955,1.224034,0.188763,0.273635,0.184514,0.324573,0.224034
1,0.5,1.0,1.0,20,1.335307,1.103426,1.360066,1.137182,1.039962,1.107587,0.137182,0.335307,0.103426,0.360066,0.107587
2,0.5,1.0,1.0,50,1.270605,1.007795,1.266871,1.039387,0.985449,1.026629,0.039387,0.270605,0.007795,0.266871,0.026629


In [11]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,1.0,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
def std_parenth_padder(type_col, col_to_pad):
    if type_col == 'std':
        return('(' + col_to_pad + ')')
    else:
        return(col_to_pad)

def format_level_0(type_col, col_to_format):
    if type_col == 'std':
        return('')
    else:
        return(col_to_format)

In [52]:
import regex as re

# Pass this function a pandas dataframe of which to create the LaTeX file
# dataframe is the df
# filename is the save location
# index tells if you want the index saved to the file or not
def Minimal_Latex(dataframe, filename, index = False):

    # Open file
    with open(filename, "w") as f:
        
        # Start from the pandas
        corrected_table = dataframe.to_latex(header = False, index = index)

        # Strip all material that pandas inserts- the tabular environment and toprule and bottomrule
        corrected_table = re.sub(r'\\begin.+', '', corrected_table)
        corrected_table = re.sub(r'\\toprule', '', corrected_table)
        corrected_table = re.sub(r'\\bottomrule', '', corrected_table)
        corrected_table = re.sub(r'\\end.+', '', corrected_table)

        # Write to file and skip blank lines
        f.write(corrected_table.strip())

In [76]:
coef_names_mapper = {'mismeasured_coef':'Single Measurement', 'mismeasured_allvar_coef':'All Measurements', 'pca_coef':'PCA', 'mismeasured_avg_coef':'Average of Measurements', 'iv_coef':'Instrumental Variable'}
ape_names_mapper = {'mismeasured_coef_ape':'Single Measurement', 'mismeasured_allvar_coef_ape':'All Measurements', 'pca_ape':'PCA', 'mismeasured_avg_coef_ape':'Average of Measurements', 'iv_coef_ape':'Instrumental Variable'}

In [57]:
sim_reg_p = (output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')]
                   .drop(columns=['covariance', 'beta1', 'beta2'])
                   .groupby('p')
                   .agg(['mean', 'std'])
                   .transpose()
                   .reset_index()
                   .rename(columns = {5:'five', 20:'twenty', 50:'fifty'}))

sim_reg_p

p,level_0,level_1,five,twenty,fifty
0,mismeasured_coef,mean,1.273635,1.335307,1.270605
1,mismeasured_coef,std,NaN,NaN,NaN
2,mismeasured_allvar_coef,mean,1.184514,1.103426,1.007795
3,mismeasured_allvar_coef,std,NaN,NaN,NaN
4,mismeasured_avg_coef,mean,1.324573,1.360066,1.266871
...,...,...,...,...,...
17,mismeasured_allvar_coef_ape,std,NaN,NaN,NaN
18,mismeasured_avg_coef_ape,mean,0.324573,0.360066,0.266871
19,mismeasured_avg_coef_ape,std,NaN,NaN,NaN
20,iv_coef_ape,mean,0.224034,0.107587,0.026629


In [58]:
sim_reg_p_coefs = (sim_reg_p.query('level_0.str.contains("coef")')
                            .query('level_0 != "true_val_coef"')
                            .query('not level_0.str.contains("ape")')
                            .round(3)
                            .astype(str))

sim_reg_p_coefs['five'] = sim_reg_p_coefs.apply(lambda x: std_parenth_padder(x.level_1, x.five), axis = 1)
sim_reg_p_coefs['twenty'] = sim_reg_p_coefs.apply(lambda x: std_parenth_padder(x.level_1, x.twenty), axis = 1)
sim_reg_p_coefs['fifty'] = sim_reg_p_coefs.apply(lambda x: std_parenth_padder(x.level_1, x.fifty), axis = 1)

sim_reg_p_coefs['level_0'] = sim_reg_p_coefs.apply(lambda x: format_level_0(x.level_1, x.level_0), axis = 1)

sim_reg_p_coefs = (sim_reg_p_coefs.drop(columns = 'level_1')
                                  .replace(coef_names_mapper))

Minimal_Latex(sim_reg_p_coefs, 'p_coefs.tex')

sim_reg_p_coefs

p,level_0,five,twenty,fifty
0,Single Measurement,1.274,1.335,1.271
1,,(nan),(nan),(nan)
2,All Measurements,1.185,1.103,1.008
3,,(nan),(nan),(nan)
4,Average of Measurements,1.325,1.36,1.267
5,,(nan),(nan),(nan)
6,PCA,1.189,1.137,1.039
7,,(nan),(nan),(nan)
10,Instrumental Variable,1.224,1.108,1.027
11,,(nan),(nan),(nan)


In [84]:
sim_reg_p_apes = (sim_reg_p.query('level_0.str.contains("ape")')
                           .query('level_1 != "std"')
                           .drop(columns = 'level_1')
                           .replace(ape_names_mapper)
                           .set_index('level_0')
                           .multiply(100)
                           .round(1)
                           .astype(str)
                           .add('%')
                           .reset_index())

Minimal_Latex(sim_reg_p_apes, 'p_apes.tex')

sim_reg_p_apes

p,level_0,five,twenty,fifty
0,PCA,18.9%,13.7%,3.9%
1,Single Measurement,27.4%,33.5%,27.1%
2,All Measurements,18.5%,10.3%,0.8%
3,Average of Measurements,32.5%,36.0%,26.7%
4,Instrumental Variable,22.4%,10.8%,2.7%


In [12]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.392676,0.297068,0.482867,0.347941,0.126243,0.362740,2.479407,2.926759,1.970682,3.828672,2.627404
1,0.5,1.0,1.0,5,1.273635,1.184514,1.324573,1.188763,0.990955,1.224034,0.188763,0.273635,0.184514,0.324573,0.224034
2,0.5,10.0,1.0,5,10.278445,10.178115,10.397776,10.237330,9.982137,10.243914,0.023733,0.027845,0.017811,0.039778,0.024391


In [13]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,10.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,0.994095,0.983749,0.992905,0.973718,0.975325,0.979110,0.026282,0.005905,0.016251,0.007095,0.020890
1,0.5,1.0,1.0,5,1.273635,1.184514,1.324573,1.188763,0.990955,1.224034,0.188763,0.273635,0.184514,0.324573,0.224034
2,0.5,1.0,10.0,5,3.913515,2.907110,4.573856,2.977885,1.040922,3.442474,1.977885,2.913515,1.907110,3.573856,2.442474


In [15]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,10.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.306442,0.431522,0.210874,0.386974,1.108455,0.356959,0.613026,0.693558,0.568478,0.789126,0.643041
1,-0.5,1.0,1.0,5,0.688088,0.809595,0.591318,0.787940,0.991823,0.753843,0.212060,0.311912,0.190405,0.408682,0.246157
2,0.0,1.0,1.0,5,0.981324,0.981242,1.000192,0.989517,0.992666,0.983436,0.010483,0.018676,0.018758,0.000192,0.016564
3,0.5,1.0,1.0,5,1.273635,1.184514,1.324573,1.188763,0.990955,1.224034,0.188763,0.273635,0.184514,0.324573,0.224034
4,0.9,1.0,1.0,5,1.729860,1.592228,1.805877,1.630148,0.959090,1.682543,0.630148,0.729860,0.592228,0.805877,0.682543


In [17]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,no,1.302670,1.121081,1.120034,1.119961,1.021782,1.141248,0.119961,0.302670,0.121081,0.120034,0.141248
1,0.5,1.0,1.0,20,no,1.299725,1.017953,1.019209,1.019431,0.984457,1.038881,0.019431,0.299725,0.017953,0.019209,0.038881
2,0.5,1.0,1.0,50,no,1.298960,1.015811,1.015819,1.015167,1.004743,1.041875,0.015167,0.298960,0.015811,0.015819,0.041875


In [19]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,20,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,1.0,50,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.367448,0.170904,0.171895,0.171929,0.053409,0.196630,0.719292,2.674484,0.709036,0.718951,0.966303
1,0.5,1.0,1.0,5,1.302670,1.121081,1.120034,1.119961,1.021782,1.141248,0.119961,0.302670,0.121081,0.120034,0.141248
2,0.5,10.0,1.0,5,10.298414,10.105304,10.105292,10.105762,9.997272,10.121088,0.010576,0.029841,0.010530,0.010529,0.012109


In [21]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,10.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,0.997181,0.988845,0.989324,0.989386,0.976645,0.994704,0.010614,0.002819,0.011155,0.010676,0.005296
1,0.5,1.0,1.0,5,1.302670,1.121081,1.120034,1.119961,1.021782,1.141248,0.119961,0.302670,0.121081,0.120034,0.141248
2,0.5,1.0,10.0,5,3.778286,1.949708,1.954646,1.955103,0.996103,2.235823,0.955103,2.778286,0.949708,0.954646,1.235823


In [23]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,10.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.217245,0.477989,0.477303,0.477019,0.952746,0.422875,0.522981,0.782755,0.522011,0.522697,0.577125
1,-0.5,1.0,1.0,5,0.724498,0.920783,0.919703,0.918035,1.012534,0.904684,0.081965,0.275502,0.079217,0.080297,0.095316
2,0.0,1.0,1.0,5,1.005180,1.001407,1.000276,1.000394,1.009075,1.000825,0.000394,0.005180,0.001407,0.000276,0.000825
3,0.5,1.0,1.0,5,1.302670,1.121081,1.120034,1.119961,1.021782,1.141248,0.119961,0.302670,0.121081,0.120034,0.141248
4,0.9,1.0,1.0,5,1.813492,1.456942,1.457083,1.457555,1.000043,1.501181,0.457555,0.813492,0.456942,0.457083,0.501181


In [25]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
